In [21]:
import torch.nn as nn
import torch.nn.functional as F
import torch
from torchvision.datasets import MNIST
from torchvision import transforms
import torch.optim as optim
from torchvision.datasets import CIFAR10

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

Device: cuda


In [22]:
# 하이퍼파라미터 설정
num_epochs = 100
learning_rate = 0.1
dropout_prob = 0.5

In [23]:
# 가중치 드랍아웃 레이어
class WeightDropout(nn.Module):
    def __init__(self, module, p=0.5):
        super(WeightDropout, self).__init__()
        self.module = module
        self.p = p
        self.weight = nn.Parameter(module.weight.data)

    def forward(self, *args):
        if self.training:
            # dropout을 적용한 가중치
            weight = F.dropout(self.weight, p=self.p, training=True)
            # self.module.weight에 새로운 가중치 할당
            self.module.weight = nn.Parameter(weight)
        return self.module(*args)

# ***Basic Code***

In [13]:
def dataset_MNIST():
  transform = transforms.Compose([transforms.ToTensor(),
                                  transforms.Normalize((0.5,), (0.5,))])
  train_dataset = MNIST(root='./data', train=True, download=True, transform=transform)
  test_dataset = MNIST(root='./data', train=False, download=True, transform=transform)
  train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64)
  test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64)
  return train_loader, test_loader


def dataset_CIFAR10():
  transform = transforms.Compose([
      transforms.Resize((32, 32)),
      transforms.ToTensor(),
      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
  ])
  train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
  test_dataset = CIFAR10(root='./data', train=False, download=True, transform=transform)
  train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64)
  test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64)
  return train_loader, test_loader

In [14]:
# 모델 학습 함수
def train(num_epochs):
  model.to(device)
  for epoch in range(num_epochs):
      model.train()
      for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

      model.eval()
      with torch.no_grad():
          correct = 0
          total = 0
          for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
          accuracy = 100 * correct / total

      print(f'Epoch [{epoch+1}/{num_epochs}], Test Accuracy: {accuracy:.2f}%')

In [ ]:
# 그래프
def plot(method, dataset, exp_num, save=False):
    figname = f"{method} ({dataset}) #{exp_num}.png"
    plt.figure(figsize=(10, 6), dpi=1000)
    plt.plot(train_acc_list, label='Train_acc')
    plt.plot(test_acc_list, label='Test_acc')
    plt.plot(train_loss_list, label='Train_loss')
    plt.plot(test_loss_list, label='Test_loss')
    plt.legend()
    plt.title('Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    if save: plt.savefig(figname)
    plt.show()

# ***Experiments***

## 1) [MNIST] Basic setting (without Dropout)

In [15]:
class Net_MNIST(nn.Module):
    def __init__(self):
        super(Net_MNIST, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10) 

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 모델 초기화
model = Net_MNIST()

# 손실 함수와 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# 데이터셋 로드
train_loader, test_loader = dataset_MNIST()

# 모델 학습
train(num_epochs)

Epoch [1/100], Test Accuracy: 97.41%
Epoch [2/100], Test Accuracy: 98.74%
Epoch [3/100], Test Accuracy: 98.90%
Epoch [4/100], Test Accuracy: 98.96%
Epoch [5/100], Test Accuracy: 98.89%
Epoch [6/100], Test Accuracy: 98.76%
Epoch [7/100], Test Accuracy: 98.98%
Epoch [8/100], Test Accuracy: 98.89%
Epoch [9/100], Test Accuracy: 98.73%
Epoch [10/100], Test Accuracy: 98.82%
Epoch [11/100], Test Accuracy: 98.91%
Epoch [12/100], Test Accuracy: 98.90%
Epoch [13/100], Test Accuracy: 98.81%
Epoch [14/100], Test Accuracy: 98.87%
Epoch [15/100], Test Accuracy: 98.86%
Epoch [16/100], Test Accuracy: 98.91%
Epoch [17/100], Test Accuracy: 99.05%
Epoch [18/100], Test Accuracy: 99.11%
Epoch [19/100], Test Accuracy: 99.15%
Epoch [20/100], Test Accuracy: 99.16%
Epoch [21/100], Test Accuracy: 99.21%
Epoch [22/100], Test Accuracy: 99.17%
Epoch [23/100], Test Accuracy: 99.18%
Epoch [24/100], Test Accuracy: 99.18%
Epoch [25/100], Test Accuracy: 99.19%
Epoch [26/100], Test Accuracy: 99.20%
Epoch [27/100], Test 

## 2) [MNIST] Node Dropout setting

In [16]:
class Net_MNIST(nn.Module):
    def __init__(self):
        super(Net_MNIST, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.dropout = nn.Dropout(dropout_prob)
        self.fc2 = nn.Linear(500, 10) 
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# 모델 초기화
model = Net_MNIST()

# 손실 함수와 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# 데이터셋 로드
train_loader, test_loader = dataset_MNIST()

# 모델 학습
train(num_epochs)

Epoch [1/100], Test Accuracy: 97.98%
Epoch [2/100], Test Accuracy: 98.84%
Epoch [3/100], Test Accuracy: 99.03%
Epoch [4/100], Test Accuracy: 99.01%
Epoch [5/100], Test Accuracy: 99.05%
Epoch [6/100], Test Accuracy: 99.20%
Epoch [7/100], Test Accuracy: 99.19%
Epoch [8/100], Test Accuracy: 99.31%
Epoch [9/100], Test Accuracy: 99.22%
Epoch [10/100], Test Accuracy: 99.31%
Epoch [11/100], Test Accuracy: 99.26%
Epoch [12/100], Test Accuracy: 99.21%
Epoch [13/100], Test Accuracy: 99.18%
Epoch [14/100], Test Accuracy: 99.30%
Epoch [15/100], Test Accuracy: 99.34%
Epoch [16/100], Test Accuracy: 99.32%
Epoch [17/100], Test Accuracy: 99.36%
Epoch [18/100], Test Accuracy: 99.22%
Epoch [19/100], Test Accuracy: 99.26%
Epoch [20/100], Test Accuracy: 99.38%
Epoch [21/100], Test Accuracy: 99.31%
Epoch [22/100], Test Accuracy: 99.28%
Epoch [23/100], Test Accuracy: 99.29%
Epoch [24/100], Test Accuracy: 99.28%
Epoch [25/100], Test Accuracy: 99.27%
Epoch [26/100], Test Accuracy: 99.24%
Epoch [27/100], Test 

## 3) [MNIST] Weight Dropout setting

In [17]:
class Net_MNIST(nn.Module):
    def __init__(self):
        super(Net_MNIST, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = WeightDropout(nn.Linear(500, 10), p=dropout_prob)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 모델 초기화
model = Net_MNIST()

# 손실 함수와 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# 데이터셋 로드
train_loader, test_loader = dataset_MNIST()

# 모델 학습
train(num_epochs)

Epoch [1/100], Test Accuracy: 97.24%
Epoch [2/100], Test Accuracy: 97.71%
Epoch [3/100], Test Accuracy: 98.61%
Epoch [4/100], Test Accuracy: 98.32%
Epoch [5/100], Test Accuracy: 98.69%
Epoch [6/100], Test Accuracy: 98.69%
Epoch [7/100], Test Accuracy: 98.89%
Epoch [8/100], Test Accuracy: 98.93%
Epoch [9/100], Test Accuracy: 98.94%
Epoch [10/100], Test Accuracy: 98.96%
Epoch [11/100], Test Accuracy: 98.76%
Epoch [12/100], Test Accuracy: 98.84%
Epoch [13/100], Test Accuracy: 98.91%
Epoch [14/100], Test Accuracy: 99.23%
Epoch [15/100], Test Accuracy: 99.20%
Epoch [16/100], Test Accuracy: 99.26%
Epoch [17/100], Test Accuracy: 99.16%
Epoch [18/100], Test Accuracy: 99.28%
Epoch [19/100], Test Accuracy: 98.96%
Epoch [20/100], Test Accuracy: 99.25%
Epoch [21/100], Test Accuracy: 99.21%
Epoch [22/100], Test Accuracy: 99.14%
Epoch [23/100], Test Accuracy: 99.05%
Epoch [24/100], Test Accuracy: 99.08%
Epoch [25/100], Test Accuracy: 99.23%
Epoch [26/100], Test Accuracy: 99.19%
Epoch [27/100], Test 

## 1) [CIFAR-10] Basic setting (without Dropout)

In [18]:
class Net_CIFAR10(nn.Module):
    def __init__(self):
        super(Net_CIFAR10, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 5, 1)
        self.conv2 = nn.Conv2d(64, 128, 5, 1)
        self.fc1 = nn.Linear(5*5*128, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 5*5*128)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 모델 초기화
model = Net_CIFAR10()

# 손실 함수와 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# 데이터셋 로드
train_loader, test_loader = dataset_CIFAR10()

# 모델 학습
train(num_epochs)

Files already downloaded and verified
Files already downloaded and verified
Epoch [1/100], Test Accuracy: 35.74%
Epoch [2/100], Test Accuracy: 57.02%
Epoch [3/100], Test Accuracy: 64.50%
Epoch [4/100], Test Accuracy: 65.51%
Epoch [5/100], Test Accuracy: 71.01%
Epoch [6/100], Test Accuracy: 72.52%
Epoch [7/100], Test Accuracy: 62.92%
Epoch [8/100], Test Accuracy: 72.62%
Epoch [9/100], Test Accuracy: 71.62%
Epoch [10/100], Test Accuracy: 70.91%
Epoch [11/100], Test Accuracy: 72.50%
Epoch [12/100], Test Accuracy: 72.19%
Epoch [13/100], Test Accuracy: 73.74%
Epoch [14/100], Test Accuracy: 72.57%
Epoch [15/100], Test Accuracy: 73.64%
Epoch [16/100], Test Accuracy: 73.04%
Epoch [17/100], Test Accuracy: 73.57%
Epoch [18/100], Test Accuracy: 71.34%
Epoch [19/100], Test Accuracy: 73.33%
Epoch [20/100], Test Accuracy: 73.76%
Epoch [21/100], Test Accuracy: 73.46%
Epoch [22/100], Test Accuracy: 73.95%
Epoch [23/100], Test Accuracy: 74.25%
Epoch [24/100], Test Accuracy: 74.15%
Epoch [25/100], Test 

## 2) [CIFAR-10] Node Dropout setting

In [19]:
class Net_CIFAR10(nn.Module):
    def __init__(self):
        super(Net_CIFAR10, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 5, 1)
        self.conv2 = nn.Conv2d(64, 128, 5, 1)
        self.fc1 = nn.Linear(5*5*128, 512)
        self.dropout = nn.Dropout(dropout_prob)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 5*5*128)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# 모델 초기화
model = Net_CIFAR10()

# 손실 함수와 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# 데이터셋 로드
train_loader, test_loader = dataset_CIFAR10()

# 모델 학습
train(num_epochs)

Files already downloaded and verified
Files already downloaded and verified
Epoch [1/100], Test Accuracy: 36.38%
Epoch [2/100], Test Accuracy: 55.32%
Epoch [3/100], Test Accuracy: 58.86%
Epoch [4/100], Test Accuracy: 69.20%
Epoch [5/100], Test Accuracy: 66.69%
Epoch [6/100], Test Accuracy: 72.94%
Epoch [7/100], Test Accuracy: 72.91%
Epoch [8/100], Test Accuracy: 73.93%
Epoch [9/100], Test Accuracy: 74.55%
Epoch [10/100], Test Accuracy: 74.63%
Epoch [11/100], Test Accuracy: 74.82%
Epoch [12/100], Test Accuracy: 73.35%
Epoch [13/100], Test Accuracy: 75.41%
Epoch [14/100], Test Accuracy: 75.19%
Epoch [15/100], Test Accuracy: 75.14%
Epoch [16/100], Test Accuracy: 72.83%
Epoch [17/100], Test Accuracy: 74.49%
Epoch [18/100], Test Accuracy: 75.27%
Epoch [19/100], Test Accuracy: 75.34%
Epoch [20/100], Test Accuracy: 71.99%
Epoch [21/100], Test Accuracy: 74.92%
Epoch [22/100], Test Accuracy: 74.38%
Epoch [23/100], Test Accuracy: 75.61%
Epoch [24/100], Test Accuracy: 74.20%
Epoch [25/100], Test 

## 3) [CIFAR-10] Weight Dropout setting 

In [20]:
class Net_CIFAR10(nn.Module):
    def __init__(self):
        super(Net_CIFAR10, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 5, 1)
        self.conv2 = nn.Conv2d(64, 128, 5, 1)
        self.fc1 = nn.Linear(5*5*128, 512)
        self.fc2 = WeightDropout(nn.Linear(512, 10), p=dropout_prob)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 5*5*128)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 모델 초기화
model = Net_CIFAR10()

# 손실 함수와 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# 데이터셋 로드
train_loader, test_loader = dataset_CIFAR10()

# 모델 학습
train(num_epochs)

Files already downloaded and verified
Files already downloaded and verified
Epoch [1/100], Test Accuracy: 37.27%
Epoch [2/100], Test Accuracy: 51.22%
Epoch [3/100], Test Accuracy: 57.01%
Epoch [4/100], Test Accuracy: 64.70%
Epoch [5/100], Test Accuracy: 60.41%
Epoch [6/100], Test Accuracy: 65.80%
Epoch [7/100], Test Accuracy: 68.53%
Epoch [8/100], Test Accuracy: 70.90%
Epoch [9/100], Test Accuracy: 71.43%
Epoch [10/100], Test Accuracy: 72.00%
Epoch [11/100], Test Accuracy: 73.13%
Epoch [12/100], Test Accuracy: 72.73%
Epoch [13/100], Test Accuracy: 72.45%
Epoch [14/100], Test Accuracy: 72.23%
Epoch [15/100], Test Accuracy: 73.11%
Epoch [16/100], Test Accuracy: 72.17%
Epoch [17/100], Test Accuracy: 73.43%
Epoch [18/100], Test Accuracy: 73.28%
Epoch [19/100], Test Accuracy: 72.94%
Epoch [20/100], Test Accuracy: 73.32%
Epoch [21/100], Test Accuracy: 73.81%
Epoch [22/100], Test Accuracy: 72.43%
Epoch [23/100], Test Accuracy: 72.53%
Epoch [24/100], Test Accuracy: 73.85%
Epoch [25/100], Test 